In [1]:
!apt-get update 1>/dev/null 2>/dev/null
!apt-get install chromium-chromedriver 1>/dev/null 2>/dev/null
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium 1>/dev/null 2>/dev/null

In [2]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import math
from bs4 import BeautifulSoup
from selenium import webdriver

In [3]:
BASE_URL = 'https://db.netkeiba.com/'
endpoint = '?pid=race_search_detail'

In [4]:
!pip install tqdm
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
from tqdm import tqdm_notebook as tqdm
import re

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [6]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# /usr/lib/chromium-browser/chromedriver
driver = webdriver.Chrome('chromedriver', options=options)
driver.implicitly_wait(3)
driver.get(BASE_URL + endpoint)

### 以下の項目で収穫対象をunコメントしてください。

## 競走種別
driver.find_element_by_id('check_track_1').click() # 芝
driver.find_element_by_id('check_track_2').click() # ダート
driver.find_element_by_id('check_track_3').click() # 障害

## 期間
links_num = driver.find_element_by_name('list')
nums_element = Select(links_num)
# nums_element.select_by_value("20")
# nums_element.select_by_value("50")
nums_element.select_by_value("100")

## 競馬場
# driver.find_element_by_id('check_Jyo_01').click() # 札幌
# driver.find_element_by_id('check_Jyo_02').click() # 函館
# driver.find_element_by_id('check_Jyo_03').click() # 福島
# driver.find_element_by_id('check_Jyo_04').click() # 新潟
driver.find_element_by_id('check_Jyo_05').click() # 東京
# driver.find_element_by_id('check_Jyo_06').click() # 中山
# driver.find_element_by_id('check_Jyo_07').click() # 中京
# driver.find_element_by_id('check_Jyo_08').click() # 京都
# driver.find_element_by_id('check_Jyo_09').click() # 阪神
# driver.find_element_by_id('check_Jyo_10').click() # 小倉

## 馬場状態
# driver.find_element_by_id('check_baba_1').click() # 良
# driver.find_element_by_id('check_baba_2').click() # 稍重
# driver.find_element_by_id('check_baba_3').click() # 重
# driver.find_element_by_id('check_baba_4').click() # 不良

## 条件
# driver.find_element_by_id('check_jyoken_1').click() # 牝馬限定
# driver.find_element_by_id('check_jyoken_2').click() # 父内国産限定
# driver.find_element_by_id('check_jyoken_3').click() # アラブ
# driver.find_element_by_id('check_jyoken_4').click() # 指定
# driver.find_element_by_id('check_jyoken_5').click() # 混合
# driver.find_element_by_id('check_jyoken_6').click() # 特指
# driver.find_element_by_id('check_jyoken_7').click() # 国際

## 馬齢
# driver.find_element_by_id('check_barei_11').click() # ２歳
# driver.find_element_by_id('check_barei_12').click() # ３歳
# driver.find_element_by_id('check_barei_13').click() # ３歳以上
# driver.find_element_by_id('check_barei_14').click() # ４歳以上

## クラス
# driver.find_element_by_id('check_grade_1').click() # G1
# driver.find_element_by_id('check_grade_2').click() # G2
# driver.find_element_by_id('check_grade_3').click() # G3
# driver.find_element_by_id('check_grade_4').click() # OP
# driver.find_element_by_id('check_grade_5').click() # 1600万
# driver.find_element_by_id('check_grade_6').click() # 1000万
# driver.find_element_by_id('check_grade_7').click() # 500万
# driver.find_element_by_id('check_grade_8').click() # 新馬
# driver.find_element_by_id('check_grade_9').click() # 未勝利
# driver.find_element_by_id('check_grade_10').click() # 未出走

## 表示件数
links_num = driver.find_element_by_name('list')
nums_element = Select(links_num)
nums_element.select_by_value("100")
driver.find_element_by_class_name('button').send_keys(Keys.ENTER)


soup = BeautifulSoup(driver.page_source, 'html.parser')
num_pattern = '\d+'
total_race_link = re.search(num_pattern, soup.find(id="contents_liquid").find(class_='pager').text.strip().replace(',', '')).group()
total_search_page = math.ceil(int(total_race_link)/100)

all_race_links = []
filename = 'out.txt'

with open(filename, mode='w') as f:
  for i in tqdm(range(1, total_search_page)):
      soup = BeautifulSoup(driver.page_source, 'html.parser')
      pattern = '/race/\d+/'
      for a in soup.find_all('a', href=True):
        if re.search(pattern, a['href']):
          f.write(a['href'].split('/')[2] + '\n')
          all_race_links.append(a['href'].split('/')[2])
      driver.execute_script("paging('{}')".format(i+1))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [7]:
ls

out.txt  sample_data/


In [8]:
import requests
import math
from bs4 import BeautifulSoup

class race:

    BASE_URL = 'https://race.netkeiba.com/'

    def __init__(self, race_endpoints=[]):
        race_endpoints = race_endpoints

    def fetch(self, race_id):
        html = requests.get('https://race.netkeiba.com/race/result.html?race_id=' + race_id + '&rf=race_submenu')
        return BeautifulSoup(html.content, 'lxml')

    def race_result(self, race_id):
        soup = self.fetch(race_id)
        result_table_soup = soup.find("table", {"class": 'RaceTable01 RaceCommon_Table ResultRefund Table_Show_All'})
        col_num = 15
        rows = []
        row = []
        for col in result_table_soup.find("tbody").findAll(["tr", "td"]):
            if (list(set(['FirstDisplay', 'HorseList']) & set(col['class']))):
                continue
            row.append(col.text.strip())
            # TODO
            # ここで騎手と馬のクロールを流す
            # if (col['class'] == ['Jockey']):
            # print(col.find('a')['href'])
            # if (col['class'] == ['Horse_Info']) and (col.find('a') != None):
            # print(col.find('a')['href'])
            if (col['class'] == ['Weight']):
                rows.append(row)
                row = []
        return rows

    def payback(self, race_id):
        soup = self.fetch(race_id)
        pay_table_souplen = soup.findAll("table", {"class": 'Payout_Detail_Table'})
        paybacks = []
        for sub_pay_table in pay_table_souplen:
            for col in sub_pay_table.find("tbody").findAll(["tr", "td"]):
                if (col['class'] == ['Tansho']):
                    paybacks.append(col.find(class_='Payout').text.replace(',', '').strip('円'))
                if (col['class'] == ['Fukusho']):
                    paybacks.extend(col.find(class_='Payout').text.replace(',', '').strip().split('円')[:3])
                if (col['class'] == ['Wakuren']):
                    paybacks.append(col.find(class_='Payout').text.replace(',', '').strip('円'))
                if (col['class'] == ['Umaren']):
                    paybacks.append(col.find(class_='Payout').text.replace(',', '').strip('円'))
                if (col['class'] == ['Wide']):
                    paybacks.extend(col.find(class_='Payout').text.replace(',', '').strip().split('円')[:3])
                if (col['class'] == ['Umatan']):
                    paybacks.append(col.find(class_='Payout').text.replace(',', '').strip('円'))
                if (col['class'] == ['Fuku3']):
                    paybacks.append(col.find(class_='Payout').text.replace(',', '').strip('円'))
                if (col['class'] == ['Tan3']):
                    paybacks.append(col.find(class_='Payout').text.replace(',', '').strip('円'))
        return paybacks

    def raptime(self, race_id):
        soup = self.fetch(race_id)
        raptime_table_souplen = soup.find("table", {"class": 'RaceCommon_Table Race_HaronTime'})
        raptimes_list = ['100m', '200m', '300m', '400m', '500m', '600m', '700m', '800m', '900m',
                         '1000m', '1100m', '1200m', '1300m', '1400m', '1500m', '1600m', '1700m',
                         '1800m', '1900m', '2000m', '2100m', '2200m', '2300m', '2400m', '2500m', '2600m']
        kyoriS = list(map(lambda y: y.text, raptime_table_souplen.find(class_='Header').findAll('th')))
        timeS = list(map(lambda y: y.text, raptime_table_souplen.find(class_='HaronTime').findAll('td')))
        raptimes = [None] * len(raptimes_list)
        for kyori, time in zip(kyoriS, timeS):
            raptimes[raptimes_list.index(kyori)] = time
        return raptimes

In [11]:
racer = race()
race_id = '202009040202'
print(racer.race_result(race_id), "\n")
print(racer.payback(race_id), "\n")
print(racer.raptime(race_id), "\n")

[['1', '1', '1', 'カケル', '牡3', '56.0', '幸', '1:35.9', '', '2', '4.2', '36.3', '4-5', '栗東大根田', '482(-4)'], ['2', '8', '17', 'アドマイヤパンドラ', '牝3', '54.0', '浜中', '1:36.0', '1/2', '10', '43.6', '35.4', '16-15', '栗東須貝', '452(+8)'], ['3', '8', '16', 'ワールドスケール', '牡3', '56.0', '小崎', '1:36.2', '1.1/4', '3', '5.5', '36.8', '2-2', '栗東野中', '466(-8)'], ['4', '8', '15', 'ガレットデロワ', '牝3', '53.0', '☆富田', '1:36.2', 'クビ', '11', '43.9', '36.1', '12-9', '栗東池添兼', '402(-4)'], ['5', '5', '10', 'レッドエーデル', '牝3', '54.0', '酒井', '1:37.1', '5', '9', '39.9', '36.8', '14-11', '栗東斉藤崇', '448(0)'], ['6', '4', '8', 'レヴィオーサ', '牡3', '56.0', '和田', '1:37.3', '1.1/4', '4', '13.4', '37.9', '9-2', '栗東友道', '450(0)'], ['7', '7', '14', 'サンテルノ', '牡3', '56.0', '北村友', '1:37.3', 'クビ', '8', '38.5', '37.5', '9-7', '栗東池添学', '514(+12)'], ['8', '4', '7', 'パズル', '牡3', '56.0', '藤井勘', '1:37.5', '1', '6', '20.1', '37.8', '5-5', '栗東池江', '490(-4)'], ['9', '7', '13', 'ハーランズハーツ', '牡3', '56.0', '松若風', '1:37.8', '1.3/4', '7', '25.5', '38.0', '5-7', '栗東橋